## Prime Sieve variants

In [1]:
#Original version
import  math
def sieve_primes(n):
    a = [True for x in range(n + 1)]
    i = 2
    while i <= math.sqrt(n):
        if a[i]:
            for j in range(i*i, n + 1, i):
                a[j] = False
        i += 1
    return [i for i in range(2, len(a)) if a[i]]

In [2]:
#cython version
%load_ext Cython

In [3]:
%%cython
import math
cpdef list cython_sieve(int n):
    cdef int i, j
    cdef list a 
    a = [True for x in range(n + 1)]
    i = 2
    while i <= math.sqrt(n):
        if a[i]:
            for j in range(i*i, n + 1, i):
                a[j] = False
        i += 1
    return [i for i in range(2, len(a)) if a[i]]

In [4]:
#numpy version
import numpy as np
def numpy_sieve(n):
    a = np.ones(n)
    a[0] = 0
    a[1] = 0
    for i in range(2,int(math.sqrt(n))):
         a[i*i::i] = 0
    return np.flatnonzero(a)

In [5]:
#numba version
from numba import jit

@jit
def numba_sieve(n):
    a = [True for x in range(n + 1)]
    i = 2
    while i <= math.sqrt(n):
        if a[i]:
            for j in range(i*i, n + 1, i):
                a[j] = False
        i += 1
    return [i for i in range(2, len(a)) if a[i]]

In [6]:
#numba version with nogil=True
from numba import jit

@jit(nogil=True)
def numba_nogil_sieve(n):
    a = [True for x in range(n + 1)]
    i = 2
    while i <= math.sqrt(n):
        if a[i]:
            for j in range(i*i, n + 1, i):
                a[j] = False
        i += 1
    return [i for i in range(2, len(a)) if a[i]]

In [7]:
#numba parallel version
from numba import prange
@jit(parallel=True)
def numba_parallel_sieve(n):
    a = [True for x in range(n + 1)]
    max = int(math.sqrt(n))
    for i in prange(2,max):
        if a[i]:
            for j in range(i*i, n + 1, i):
                a[j] = False
    return [i for i in range(2, len(a)) if a[i]]

In [8]:
#numba parallel version with nogil=True
from numba import prange
@jit(parallel=True,nogil=True)
def numba_parallel_nogil_sieve(n):
    a = [True for x in range(n + 1)]
    max = int(math.sqrt(n))
    for i in prange(2,max):
        if a[i]:
            for j in range(i*i, n + 1, i):
                a[j] = False
    return [i for i in range(2, len(a)) if a[i]]

In [9]:
# Make N smaller if this takes too long to run
N=10000000

original_time = %timeit -o sieve_primes(N)
numpy_time = %timeit -o numpy_sieve(N)
cython_time = %timeit -o cython_sieve(N)
numba_time = %timeit -o numba_sieve(N)
parallel_numba_time = %timeit -o numba_parallel_sieve(N)
parallel_nogil_numba_time = %timeit -o numba_parallel_nogil_sieve(N)

3.91 s ± 690 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.03 s ± 97 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.45 s ± 130 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
399 ms ± 9.24 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
431 ms ± 10.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
403 ms ± 6.44 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
print('Numpy speed up is {0:.1f} times'.format(original_time.average/numpy_time.average))
print('Cython speed up is {0:.1f} times'.format(original_time.average/cython_time.average))
print('single-core speed up using numba is {0:.1f} times'.format(original_time.average/numba_time.average))
print('multi-core speed up using numba is {0:.1f} times'.format(original_time.average/parallel_numba_time.average))
print('multi-core and nogil speed up using numba is {0:.1f} times'.format(original_time.average/parallel_nogil_numba_time.average))

Numpy speed up is 3.8 times
Cython speed up is 2.7 times
single-core speed up using numba is 9.8 times
multi-core speed up using numba is 9.1 times
multi-core and nogil speed up using numba is 9.7 times


Q: Is the pattern consistent with smaller and larger value of N?